# `MEDUSA`
aka. Dynamic-Prospective LCA aka. Union(premise, temporalis)

In [1]:
from bw_temporalis import easy_timedelta_distribution, TemporalDistribution, easy_datetime_distribution
import sys
import os
sys.path.append(os.path.realpath('../'))
from medusa.edge_extractor import *
from medusa.matrix_modifier import MatrixModifier
from medusa.medusa_lca import *
from medusa.timeline_builder import TimelineBuilder
import bw2data as bd
import bw2calc as bc
import numpy as np
import pandas as pd

In [2]:
from tests.databases import *
# db_abc_simple() # works
# db_abc_C_to_E() # works
# db_abc_C_and_B_to_E() # BREAKS THINGS
# db_abc_B_to_E() # BREAKS THINGS
# db_abc_B_to_E_simplified() # BREAKS THINGS
# db_abc_B_to_E_simplified_and_E_with_TD()
db_abc_loopA_with_biosphere()

100%|██████████| 1/1 [00:00<00:00, 905.90it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<00:00, 44384.17it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<00:00, 31536.12it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 4/4 [00:00<00:00, 76959.71it/s]

Vacuuming database 


In [3]:
demand = {('foreground', 'A'): 1}
gwp = ('GWP', 'example')

# Static LCA

In [4]:
slca = bc.LCA(demand, gwp)
slca.lci()
slca.lcia()
print(f'Static LCA score: {slca.score}')

Static LCA score: 240.5555559538397


# `MEDUSA` LCA

In [5]:
SKIPPABLE = [node.id for node in bd.Database('background_2020')] + [
    node.id for node in bd.Database('background_2024')
]

def filter_function(database_id: int) -> bool:
    return database_id in SKIPPABLE

Now we can do the graph traversal and create a timeline of edges:

In [6]:
import warnings
warnings.filterwarnings("ignore")

database_date_dict = {
            datetime.strptime("2008", "%Y"): 'background_2008',
            datetime.strptime("2024", "%Y"): 'background_2024',
        }

In [7]:
# tl_builder = TimelineBuilder(slca, filter_function, database_date_dict)

# timeline = tl_builder.build_timeline()

# demand_timing_dict = create_demand_timing_dict(timeline, demand) # Should go elsewhere as well - medusa LCA?

# mm = MatrixModifier(timeline, database_date_dict, demand_timing_dict)
# dp = mm.create_datapackage()

# fu, data_objs, remapping = prepare_medusa_lca_inputs(demand=demand, demand_timing_dict=demand_timing_dict, method=gwp) 
# lca = bc.LCA(fu, data_objs = data_objs + dp, remapping_dicts=remapping)
# lca.lci()
# lca.lcia()

In [8]:
mlca = MedusaLCA(slca, filter_function, database_date_dict)

Starting graph traversal
Calculation count: 42


In [9]:
mlca.build_timeline()

All databases in database_date_dict exist as brightway project databases


,hash_producer,date_producer,producer,producer_name,hash_consumer,date_consumer,consumer,consumer_name,amount,interpolation_weights
0,1995,1995-01-01,2,C,2000,2000-01-01,8,B,13.0,{'background_2008': 1}
1,1996,1996-01-01,2,C,2001,2001-01-01,8,B,13.0,{'background_2008': 1}
2,1997,1997-01-01,2,C,2002,2002-01-01,8,B,13.0,{'background_2008': 1}
3,1998,1998-01-01,2,C,2003,2003-01-01,8,B,13.0,{'background_2008': 1}
4,1999,1999-01-01,2,C,2004,2004-01-01,8,B,13.0,{'background_2008': 1}
...,...,...,...,...,...,...,...,...,...,...
79,2035,2035-01-01,8,B,2020,2020-01-01,9,A,0.4,{'background_2024': 1}
80,2036,2036-01-01,8,B,2021,2021-01-01,9,A,0.4,{'background_2024': 1}
81,2037,2037-01-01,8,B,2022,2022-01-01,9,A,0.4,{'background_2024': 1}
82,2038,2038-01-01,8,B,2023,2023-01-01,9,A,0.4,{'background_2024': 1}


In [10]:
mlca.build_datapackage()

In [11]:
mlca.lci()

In [12]:
print('New MEDUSA LCA Score:', mlca.score)
print('Old static LCA Score:', slca.score)

New MEDUSA LCA Score: None
Old static LCA Score: 240.5555559538397


In [19]:
df = pd.DataFrame(lca.technosphere_matrix.toarray())
df.rename(lca.dicts.activity.reversed, inplace=True, axis=0)
df.rename(lca.dicts.activity.reversed, inplace=True, axis=1)
df


,2,3,4,5,6,7,8,9,2002012,2002013,...,6002022,7002022,7002023,8002017,8002018,8002019,8002020,8002021,8002022,9002024
2,1.0,0.0,0.0,0.0,0.0,0.0,-13.0,0.0,-0.25,-0.312628,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
3,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.75,-0.687372,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
5,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.00,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
6,0.0,0.0,0.0,0.0,1.0,-2.0,0.0,0.0,0.00,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
7,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.5,0.00,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
8,0.0,0.0,0.0,0.0,-8.0,-5.0,1.0,0.0,0.00,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.00,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
2002012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,0.000000,...,0.0,0.0,0.0,-13.0,0.0,0.0,0.0,0.0,0.0,0.00
2002013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1.000000,...,0.0,0.0,0.0,0.0,-13.0,0.0,0.0,0.0,0.0,0.00


In [13]:
rem = lca.remapping_dicts['activity']
rem

{1: ('temporalis-bio', 'CO2'),
 2: ('background_2024', 'C'),
 3: ('background_2024', 'electricity_wind'),
 4: ('background_2008', 'C'),
 5: ('background_2008', 'electricity_wind'),
 6: ('foreground', 'E'),
 7: ('foreground', 'D'),
 8: ('foreground', 'B'),
 9: ('foreground', 'A')}

In [18]:
dict(lca.dicts.activity)

{2: 0,
 3: 1,
 4: 2,
 5: 3,
 6: 4,
 7: 5,
 8: 6,
 9: 7,
 2001999: 8,
 2002016: 9,
 2002017: 10,
 2002027: 11,
 2002028: 12,
 2002032: 13,
 2002033: 14,
 2002034: 15,
 6002022: 16,
 6002023: 17,
 7002022: 18,
 7002023: 19,
 8002004: 20,
 8002021: 21,
 8002022: 22,
 8002039: 23,
 9002024: 24}

In [15]:
abs = timeline[1].abs_td_consumer

In [17]:
abs.date

array(['2024-01-30T18:45:57'], dtype='datetime64[s]')

In [9]:
td = TemporalDistribution(
                            date=np.array(
                                [
                                    0,
                                ],
                                dtype="timedelta64[Y]",
                            ),
                            amount=np.array([1]),
                        )

In [11]:
td.date

array([0], dtype='timedelta64[s]')

In [ ]:
def join_datetime_and_timedelta_distributions(
        td_producer: TemporalDistribution, td_consumer: TemporalDistribution
    ) -> TemporalDistribution:
        """TODO: Needs description"""
        # if an edge does not have a TD, then return the consumer_td so that the timeline continues
        if isinstance(td_consumer, TemporalDistribution) and isinstance(
            td_producer, Number
        ):
            return td_consumer
        # Else, if both consumer and producer have a td (absolute and relative, respectively) join to TDs
        if isinstance(td_producer, TemporalDistribution) and isinstance(
            td_consumer, TemporalDistribution
        ):
            if not (td_consumer.date.dtype == datetime_type):
                raise ValueError(
                    f"`td_consumer.date` must have dtype `datetime64[s]`, but got `{td_consumer.date.dtype}`"
                )
            if not (td_producer.date.dtype == timedelta_type):
                raise ValueError(
                    f"`td_producer.date` must have dtype `timedelta64[s]`, but got `{td_producer.date.dtype}`"
                )
            date = (
                td_consumer.date.reshape((-1, 1)) + td_producer.date.reshape((1, -1))
            ).ravel()
            amount = np.array(len(td_consumer) * [td_producer.amount]).ravel()
            return TemporalDistribution(date, amount)
        else:
            raise ValueError(
                "Can't join TemporalDistribution and something else: Trying with {} and {}".format(
                    type(td1), type(td2)
                )
            )